# install packages

In [1]:
!pip install pyyaml==5.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.4/269.4 kB 15.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp310-cp310-linux_x86_64.whl size=44635 sha256=9564042fac9149e5744b7120873c79692e40bc5edd9b62e5fb7c15ebb4108815
  Stored in directory: /home/jovyan/.cache/pip/wheels/0b/a9/6a/d0a6981a8dbb698845178818642f72ce179f14336908c7df01
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0


In [2]:
!pip install gen3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.7/162.7 kB 11.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 21.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 19.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 74.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.6/69.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━

In [3]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 63.7 MB/s eta 0:00:00


In [25]:
import gen3
import os
import time

In [34]:
# test pulling MIDRC GUIDs -- this works
os.system("gen3 --auth midrc_credentials.json --endpoint data.midrc.org drs-pull object dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855")
          

{"succeeded": ["dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855"], "failed": []}


0

In [24]:
#!gen3 --auth midrc_credentials.json --endpoint data.midrc.org drs-pull object dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb

{"succeeded": ["dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb"], "failed": []}


# write nextflow config

In [17]:
%%writefile nextflow.config

process {
    withLabel: download_data {
        executor = 'local'
    }
    withLabel: publish_data {
        executor = 'local'
    }
    withLabel: dcm2png {
        executor = 'local'
    }
}

workDir = 'sdk_data'

Overwriting nextflow.config


# Pull some midrc guids to test and move dcm files to midrc_dicom directory

In [37]:
%%writefile download_data.py

import gen3
import os

output_dir = 'data'

guids = ['dg.MD1R/ea669b5e-ae51-40ba-b375-ed23a9cd1855',
         'dg.MD1R/a745ed98-0cb9-4537-826b-13b2e354e8bb',
         'dg.MD1R/e604979a-c71b-4ec6-b8a0-959837b86384',
         'dg.MD1R/b5cee98d-46ff-4438-aa00-90727a383340',
         'dg.MD1R/8a5a5579-7925-432d-a614-3ed208f1c182',
         'dg.MD1R/33034812-47f3-4c0e-b60b-fa7a2a04ecda',
         'dg.MD1R/5ca987c5-c660-4785-a67d-a3424cc8ec6e',
         'dg.MD1R/44148117-1858-49ef-b30f-d239abfaff80',
         'dg.MD1R/9ea205e8-a774-4318-a323-95eadda9bc5c',
         'dg.MD1R/09ece36f-a0fa-48e8-8fc2-62110eaae570']

midrc_creds_file = "/home/jovyan/pd/midrc_credentials.json"
endpoint = 'data.midrc.org'

for guid in guids:
    print('pulling guid {}'.format(guid))
    os.system("gen3 --auth {} --endpoint {} drs-pull object {}".format(midrc_creds_file, endpoint, guid))
    #os.system("time.sleep(3)")

Overwriting download_data.py


In [27]:
%%writefile convert_dcm_to_png.py


import pydicom
import argparse
import numpy as np
from PIL import Image
import os

def main(dicom_inputs):
    for dicom_file in dicom_inputs:
        png_out = dicom_file.split('.dcm')[0] + '.png'
        dicom_dataset = pydicom.dcmread(dicom_file)
        transformed_image = dicom_dataset.pixel_array.astype(float)
        scaled_image = np.uint8((np.maximum(transformed_image, 0) / transformed_image.max()) * 255.0)
        final_image = Image.fromarray(scaled_image)
        final_image.save(png_out)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='convert dicom to png')
    parser.add_argument('--dicom-inputs', dest='dicom_inputs', nargs = '+', required=True)
    args = parser.parse_args()

    main(args.dicom_inputs)

Overwriting convert_dcm_to_png.py


In [28]:
%%writefile main.nf
#!/usr/bin/env nextflow

process DownloadData {
    
    label 'download_data'
    
    tag 'initial_task'
    
    output:
    stdout emit: download_data_log
    path('**/*.dcm'), emit: dicom_image
    
    script:
    """
    python3 ${baseDir}/download_data.py
    """
}

process PublishData {
    
    label 'publish_data'
    
    publishDir "${baseDir}/dicom_files", mode: 'copy'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    
    script:
    """
    echo $dicom_image
    """
}

process convert_dcm_to_png {
    
    label 'dcm2png'
    
    input:
    path(dicom_image) 
    
    output:
    stdout emit: convert_to_png_log
    path('*.png'), emit: png_image
    
    script:
    """
    python3 ${baseDir}/convert_dcm_to_png.py --dicom-inputs $dicom_image
    """
}
// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    DownloadData()
    PublishData(DownloadData.out.dicom_image)
    convert_dcm_to_png(DownloadData.out.dicom_image)
}


Overwriting main.nf


In [41]:
!nextflow run main.nf -dsl2

N E X T F L O W  ~  version 21.10.6
Launching `main.nf` [spontaneous_wing] - revision: 19d1d0ed2e
[-        ] process > DownloadData       [  0%] 0 of 1
[-        ] process > PublishData        -
[-        ] process > convert_dcm_to_png -

executor >  local (1)
[61/3ba9b2] process > DownloadData (initial_task) [  0%] 0 of 1
[-        ] process > PublishData                 -
[-        ] process > convert_dcm_to_png          -

executor >  local (1)
[61/3ba9b2] process > DownloadData (initial_task) [100%] 1 of 1 ✔
[-        ] process > PublishData                 -
[-        ] process > convert_dcm_to_png          -

executor >  local (2)
[61/3ba9b2] process > DownloadData (initial_task) [100%] 1 of 1 ✔
[-        ] process > PublishData                 [  0%] 0 of 1
[35/7b1283] process > convert_dcm_to_png          [  0%] 0 of 1

executor >  local (3)
[61/3ba9b2] process > DownloadData (initial_task) [100%] 1 of 1 ✔
[03/9409a5] process > PublishData                 [  0%] 0 of 1
[35/7b1